In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import torch
import torch.utils.data as Data
from scipy.sparse import coo_matrix
import hickle as hkl
import os

from prog.nci_data_load import dataload
from prog.nci_data_process import process

In [2]:
ls data/Celline

Cell_lines_annotations.txt
GDSC_IC50.csv
genomic_expression_561celllines_697genes_demap_features.csv
genomic_methylation_561celllines_808genes_demap_features.csv
genomic_mutation_34673_demap_features.csv


In [3]:
# from prog.data_load import dataload
# from prog.data_process import process

In [4]:
drug_feature, mutation_feature, gexpr_feature, methylation_feature, nb_celllines,  nb_drugs, = dataload()

In [13]:
methylation_feature.dropna()

,A1BG-AS1,A1CF,A2ML1,A4GALT,A4GNT,AAA1,AAAS,AACS,AACSP1,AADACL3,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
SK_MEL_2,0.883,0.268,0.899,0.316,0.834,0.661,0.104,0.082,0.517,0.387,...,0.065,0.068,0.113,0.110,0.189,0.574,0.122,0.069,0.121,0.097
NCI_H460,0.853,0.603,0.904,0.069,0.235,0.912,0.095,0.105,0.678,0.294,...,0.049,0.071,0.187,0.141,0.130,0.333,0.104,0.081,0.109,0.086
DU_145,0.881,0.772,0.913,0.089,0.898,0.906,0.108,0.093,0.863,0.322,...,0.056,0.072,0.091,0.101,0.127,0.404,0.094,0.080,0.100,0.084


In [5]:
train = pd.read_csv("nci_data/train.csv")
train['labels'] = np.load('nci_data/train_labels.npy')
# train.to_csv('nci_data/nci_train.csv', index=None)
# train

In [6]:
val = pd.read_csv("nci_data/val.csv")
val['labels'] = np.load('nci_data/val_labels.npy')
# val.to_csv('nci_data/nci_val.csv', index=None)
# val

In [7]:
test = pd.read_csv("nci_data/test.csv")
test['labels'] = np.load('nci_data/test_labels.npy')
# test.to_csv('nci_data/nci_test.csv', index=None)
# test

In [17]:
# トレーニングデータのラベル分布を確認
print("Train labels distribution:", np.bincount(train['labels'].astype(int).values))
# テストデータのラベル分布を確認
print("Test labels distribution:", np.bincount(test['labels'].astype(int).values))

Train labels distribution: [5122 4562]
Test labels distribution: [1724 1504]


In [16]:
from 

,Drug,Cell,labels
0,682298,PC_3,0.0
1,141549,OVCAR_8,0.0
2,56410,MDA_MB_231,0.0
3,603077,U251,1.0
4,757804,IGROV1,0.0
...,...,...,...
9679,756717,NCI_ADR_RES,0.0
9680,9706,K_562,0.0
9681,123127,BT_549,0.0
9682,681646,TK_10,0.0


In [99]:
def CalculateGraphFeat(feat_mat, adj_list):
    assert feat_mat.shape[0] == len(adj_list)
    adj_mat = np.zeros((len(adj_list), len(adj_list)), dtype="float32")
    for i in range(len(adj_list)):
        nodes = adj_list[i]
        for each in nodes:
            adj_mat[i, int(each)] = 1
    assert np.allclose(adj_mat, adj_mat.T)
    x, y = np.where(adj_mat == 1)
    adj_index = np.array(np.vstack((x, y)))
    return [feat_mat, adj_index]

def FeatureExtract(drug_feature):
    drug_data = [[] for item in range(len(drug_feature))]
    for i in range(len(drug_feature)):
        feat_mat, adj_list, _ = drug_feature.iloc[i]
        drug_data[i] = CalculateGraphFeat(feat_mat, adj_list)
    return drug_data

In [101]:
pubid = [str(item[0]) for item in pubmedmap]
drug_feature = pd.DataFrame(drug_feature).T
drug_feature = drug_feature.loc[pubid]
atom_shape = drug_feature[0].iloc[0].shape[-1]
drug_data = FeatureExtract(drug_feature)

In [108]:
# ----cell line_feature_input
cellid = [item[0] for item in cellmap]
gexpr_feature = gexpr_feature.loc[cellid]
mutation_feature = mutation_feature.loc[cellid]
methylation_feature = methylation_feature.loc[cellid]
mutation = torch.from_numpy(np.array(mutation_feature, dtype="float32"))
mutation = torch.unsqueeze(mutation, dim=1)
mutation = torch.unsqueeze(mutation, dim=1)
gexpr = torch.from_numpy(np.array(gexpr_feature, dtype="float32"))
methylation = torch.from_numpy(np.array(methylation_feature, dtype="float32"))

In [112]:
# ---compile training set and test set
drug_set = Data.DataLoader(
    dataset=GraphDataset(graphs_dict=drug_data),
    collate_fn=collate,
    batch_size=nb_drugs,
    shuffle=False,
)
cellline_set = Data.DataLoader(
    dataset=Data.TensorDataset(mutation, gexpr, methylation),
    batch_size=nb_celllines,
    shuffle=False,
)

In [9]:
process(
    drug_feature,
    mutation_feature,
    gexpr_feature,
    methylation_feature,
    train,  # 分割済みのトレーニングデータ（DataFrame）
    val,  # 分割済みの検証データ（DataFrame、オプション）
    test,   # 分割済みのテストデータ（DataFrame）
    nb_celllines,
    nb_drugs,
)

(<torch.utils.data.dataloader.DataLoader at 0x345cd3520>,
 array([[ 39., 194.,  -1.],
        [ 35.,  91.,  -1.],
        [ 26., 240.,  -1.],
        ...,
        [ 69.,  59.,   1.],
        [224.,  50.,   1.],
        [ 12.,  10.,   1.]]),
 tensor([1., 0., 0.,  ..., 0., 1., 1.]),
 tensor([ True, False, False,  ..., False,  True,  True]),
 tensor([False,  True,  True,  ...,  True, False, False]),
 75,
 tensor([False, False, False,  ..., False, False, False]))

In [120]:
train

,Drug,Cell,labels
0,682298,PC_3,0.0
1,141549,OVCAR_8,0.0
2,56410,MDA_MB_231,0.0
3,603077,U251,1.0
4,757804,IGROV1,0.0
...,...,...,...
9679,756717,NCI_ADR_RES,0.0
9680,9706,K_562,0.0
9681,123127,BT_549,0.0
9682,681646,TK_10,0.0


In [17]:
# ------data files
Drug_info_file = "data/Drug/1.Drug_listMon Jun 24 09_00_55 2019.csv"
IC50_threds_file = "data/Drug/drug_threshold.txt"
Drug_feature_file = "data/Drug/drug_graph_feat"
Cell_line_info_file = "data/Celline/Cell_lines_annotations.txt"
Genomic_mutation_file = "data/Celline/genomic_mutation_34673_demap_features.csv"
Cancer_response_exp_file = "data/Celline/GDSC_IC50.csv"
Gene_expression_file = (
    "data/Celline/genomic_expression_561celllines_697genes_demap_features.csv"
)
Methylation_file = (
    "data/Celline/genomic_methylation_561celllines_808genes_demap_features.csv"
)

print("Extracting bio-features...")
# -------bio-feature extraction
(
    drug_feature,
    mutation_feature,
    gexpr_feature,
    methylation_feature,
    data_new,
    nb_celllines,
    nb_drugs,
) = dataload(
    Drug_info_file,
    IC50_threds_file,
    Drug_feature_file,
    Cell_line_info_file,
    Genomic_mutation_file,
    Cancer_response_exp_file,
    Gene_expression_file,
    Methylation_file,
)

Extracting bio-features...


/Users/inouey2/miniconda3/envs/torch/lib/python3.10/site-packages/hickle/hickle.py:356: UserWarning: Input argument 'file_obj' appears to be a file made with hickle v3. Using legacy load...
  warnings.warn("Input argument 'file_obj' appears to be a file made"


All 100572 pairs across 561 cell lines and 222 drugs.


In [21]:
nb_drugs

222

In [4]:
Drug_feature_file = 'nci_data/drug_graph_feat'
test = {}
for each in os.listdir(Drug_feature_file):
    feat_mat, adj_list, degree_list = hkl.load("%s/%s" % (Drug_feature_file, each))
    test[each.split(".")[0]] = [feat_mat, adj_list, degree_list]

In [18]:
# test

In [5]:
gexpr_feature.head()

NameError: name 'gexpr_feature' is not defined

In [6]:
exp = pd.read_csv('nci_data/nci60_gene_exp.csv', index_col=0).T
exp.head()

,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A2MP1,A3GALT2,A4GALT,A4GNT,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
MCF7,1.92,2.89,0.03,0.03,0.21,0.00,0.0,0.00,2.46,0.0,...,8.52,22.48,0.44,1.51,5.59,0.97,0.94,6.99,2.52,3.05
MDA_MB_231,0.49,0.12,0.02,0.02,0.00,0.01,0.0,0.07,0.22,0.0,...,16.76,27.56,0.32,3.43,1.44,0.55,1.58,8.15,0.45,18.57
HS578T,3.37,1.17,0.04,0.04,0.00,0.09,0.0,0.00,0.35,0.3,...,1.97,3.22,0.16,0.34,2.11,0.14,1.42,103.53,1.27,1.65
BT_549,6.00,1.92,0.00,0.00,0.00,0.00,0.0,0.00,5.52,0.0,...,1.93,3.75,0.45,0.94,2.79,0.49,1.22,36.05,2.50,2.31
T47D,3.73,1.65,0.01,0.01,0.06,0.00,0.0,0.15,0.71,0.0,...,4.02,25.63,0.55,1.85,5.18,0.80,1.51,13.00,1.64,1.28


In [7]:
mutation_feature.head()

,TNFRSF14.1:2488170,TNFRSF14.1:2489805,TNFRSF14.1:2489824,TNFRSF14.1:2489856,TNFRSF14.1:2489868,TNFRSF14.1:2491328,TNFRSF14.1:2491335,TNFRSF14.1:2491373,TNFRSF14.1:2491403,TNFRSF14.1:2492117,...,MTCP1.23:154293902,MTCP1.23:154293909,MTCP1.23:154293951,MTCP1.23:154293978,MTCP1.23:154294034,MTCP1.23:154294203,MTCP1.23:154294207,MTCP1.23:154294278,MTCP1.23:154294279,MTCP1.23:154298967
ACH-000828,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ACH-000568,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ACH-000560,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ACH-000561,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ACH-000562,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
mutation = pd.read_csv('nci_data/nci60GeneMut.csv', index_col=0).T
mutation.index = exp.index
mutation.head()

,C1orf222,SAMD11,KLHL17,PLEKHN1,ISG15,AGRN,TTLL10,TNFRSF4,UBE2J2,ACAP3,...,ATP6AP1,GDI1,FAM50A,PLXNA3,GAB3,F8,MTCP1,CLIC2,TMLHE,DDX3Y
MCF7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
MDA_MB_231,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
HS578T,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
BT_549,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
T47D,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
methylation_feature.head()

,SKI_1_2159133_2160133,TNFRSF14_1_2486803_2487803,PRDM16_1_2984741_2985741,RPL22_1_6259679_6260679,CAMTA1_1_6844383_6845383,MTOR_1_11322608_11323608,PRDM2_1_14025734_14026734,PRDM2_1_14074875_14075875,CASP9_1_15850940_15851940,CASP9_1_15851285_15852285,...,BCORL1_X_129138163_129139163,ELF4_X_129244475_129245475,ELF4_X_129244688_129245688,GPC3_X_133119673_133120673,ATP2B3_X_152800579_152801579,FLNA_X_153603006_153604006,RPL10_X_153625405_153626405,RPL10_X_153625712_153626712,RPL10_X_153625837_153626837,MTCP1_X_154299547_154300547
ACH-000698,0.00000,0.00739,0.04799,0.14687,0.00553,0.54138,0.07160,0.00040,0.01836,0.03461,...,1.00000,0.00000,0.00000,0.020070,0.01706,0.16926,0.00145,0.00242,0.00242,0.01924
ACH-000489,0.00000,0.00363,0.64891,1.00000,0.00538,0.00000,0.00000,0.00000,0.00000,0.00000,...,1.00000,0.00000,0.00000,0.320580,0.72527,0.37572,0.00000,0.00000,0.00000,0.00000
ACH-000522,0.00143,0.47465,0.27715,0.19441,0.00858,0.40555,0.21625,0.01367,0.00338,0.00338,...,1.00000,0.18059,0.22562,0.054325,0.60830,0.65268,0.82345,0.59039,0.59039,0.07546
ACH-000613,0.00000,0.00000,0.05112,0.10686,0.00268,0.28618,0.07706,0.00000,0.00000,0.00000,...,0.01066,0.00000,0.00000,0.010660,0.53086,0.00918,0.00000,0.00448,0.00448,0.00000
ACH-000614,0.00118,0.00471,0.15165,0.07407,0.01050,0.51629,0.03292,0.00405,0.00000,0.00000,...,1.00000,0.07273,0.07273,0.106990,0.69472,0.28947,0.05354,0.04306,0.04306,0.00114


In [14]:
methylation = pd.read_csv('nci_data/nci60_methylation.csv', index_col=0).T
methylation.head()

,A1BG-AS1,A1CF,A2ML1,A4GALT,A4GNT,AAA1,AAAS,AACS,AACSP1,AADACL3,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
MCF7,0.861,0.437,0.866,0.072,0.767,0.882,0.095,0.094,0.843,0.804,...,0.048,0.064,0.068,0.074,0.115,0.225,0.111,0.075,0.100,0.083
MDA_MB_231,0.861,0.727,0.704,0.077,0.492,0.886,0.134,0.101,0.726,0.185,...,0.059,0.071,0.186,0.119,0.140,0.612,0.116,0.075,0.103,0.072
HS578T,0.864,0.230,0.808,0.150,0.351,0.338,0.104,0.086,0.507,0.708,...,0.054,0.065,0.194,0.136,0.097,0.660,0.068,0.069,0.095,0.067
BT_549,0.866,0.796,0.905,0.071,0.560,0.910,0.101,0.089,0.777,0.299,...,0.065,0.068,0.120,0.105,0.138,0.167,0.086,0.079,0.099,0.068
T47D,0.863,0.241,0.909,0.074,0.179,0.550,0.094,0.091,0.624,0.266,...,0.052,0.067,0.089,0.084,0.098,0.096,0.087,0.074,0.103,0.078


[('ACH-001307', '9967941', -1, 'THCA'),
 ('ACH-001307', '9956222', -1, 'THCA'),
 ('ACH-001307', '9956119', 1, 'THCA'),
 ('ACH-001307', '9952773', -1, 'THCA'),
 ('ACH-001307', '9943465', -1, 'THCA'),
 ('ACH-001307', '9938202', -1, 'THCA'),
 ('ACH-001307', '9931953', -1, 'THCA'),
 ('ACH-001307', '9914412', -1, 'THCA'),
 ('ACH-001307', '9911830', -1, 'THCA'),
 ('ACH-001307', '9910224', -1, 'THCA'),
 ('ACH-001307', '9903786', -1, 'THCA'),
 ('ACH-001307', '9884685', -1, 'THCA'),
 ('ACH-001307', '9874913', -1, 'THCA'),
 ('ACH-001307', '9868037', -1, 'THCA'),
 ('ACH-001307', '9863776', -1, 'THCA'),
 ('ACH-001307', '9858940', -1, 'THCA'),
 ('ACH-001307', '9826528', -1, 'THCA'),
 ('ACH-001307', '9813758', -1, 'THCA'),
 ('ACH-001307', '9810884', -1, 'THCA'),
 ('ACH-001307', '84691', -1, 'THCA'),
 ('ACH-001307', '8249', -1, 'THCA'),
 ('ACH-001307', '82146', -1, 'THCA'),
 ('ACH-001307', '71271629', -1, 'THCA'),
 ('ACH-001307', '704473', -1, 'THCA'),
 ('ACH-001307', '6918848', -1, 'THCA'),
 ('ACH-0

In [13]:
drug_set, cellline_set, train_edge, label_pos, train_mask, test_mask, atom_shape = (
    process(
        drug_feature,
        mutation_feature,
        gexpr_feature,
        methylation_feature,
        data_new,
        nb_celllines,
        nb_drugs,
    )
)

In [11]:
pd.read_table('data/Celline/Cell_lines_annotations.txt').head()

,CCLE_ID,depMapID,Name,Pathology,Site_Primary,Site_Subtype1,Site_Subtype2,Site_Subtype3,Histology,Hist_Subtype1,...,Growth.Medium,Supplements,Freezing.Medium,Doubling.Time.from.Vendor,Doubling.Time.Calculated.hrs,type,type_refined,PATHOLOGIST_ANNOTATION,mutRate,tcga_code
0,DMS53_LUNG,ACH-000698,DMS 53,primary,lung,NS,NS,NS,carcinoma,small_cell_carcinoma,...,Waymouth's +10%FBS,NaN,5%DMSO,NaN,450.0,lung_small_cell,lung_small_cell,Lung:SCLC,157.241638,SCLC
1,SW1116_LARGE_INTESTINE,ACH-000489,SW1116,primary,large_intestine,colon,NS,NS,carcinoma,adenocarcinoma,...,L15+10%FBS,NaN,5%DMSO,NaN,123.2,colorectal,colorectal,Colorectal:Carcinoma,94.373192,COAD/READ
2,NCIH1694_LUNG,ACH-000431,NCI-H1694,metastasis,lung,NS,NS,NS,carcinoma,small_cell_carcinoma,...,DMEM:F12 (1:1) + 5% FBS,".005 mg/ml insulin, .01 mg/ml transferrin, 30n...",5% DMSO,NaN,81.2,lung_small_cell,lung_small_cell,Lung:SCLC,109.534987,SCLC
3,P3HR1_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,ACH-000707,P3HR-1,metastasis,haematopoietic_and_lymphoid_tissue,NS,NS,NS,lymphoid_neoplasm,Burkitt_lymphoma,...,NaN,NaN,NaN,NaN,NaN,lymphoma_Burkitt,lymphoma_Burkitt,Lymphoma:NH_B_cell,166.456406,DLBC
4,HUT78_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,ACH-000509,HuT 78,primary,haematopoietic_and_lymphoid_tissue,skin,NS,NS,lymphoid_neoplasm,mycosis_fungoides-Sezary_syndrome,...,NaN,NaN,NaN,NaN,NaN,lymphoma_other,T-cell_lymphoma_other,Lymphoma:NH_T_cell,118.217569,NaN


In [12]:
pd.read_csv('data/Celline/GDSC_IC50.csv', index_col=0).head()

,ACH-002137,ACH-000474,ACH-002089,ACH-000956,ACH-000948,ACH-000323,ACH-001002,ACH-000905,ACH-000973,ACH-000896,...,ACH-002207,ACH-000827,ACH-000534,ACH-001709,ACH-000332,ACH-000469,ACH-000570,ACH-002208,ACH-002317,ACH-000828
GDSC:1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2.724636,2.576109,NaN,NaN,NaN,NaN,2.842845,NaN
GDSC:1001,7.258918,9.131374,10.182594,8.332992,8.597157,7.606939,8.168512,8.021503,7.517325,7.215900,...,5.794405,7.748403,7.899330,6.896593,9.338678,9.200637,6.464226,NaN,9.426281,8.916867
GDSC:1004,-3.802467,-5.702659,-4.499864,-4.366928,-4.563040,-4.633045,-6.050113,-6.618734,-5.374042,-3.826083,...,-5.929886,-4.435285,-5.623723,-5.886495,-2.237481,-5.476621,-4.985852,NaN,-1.382754,-2.206867
GDSC:1005,4.146364,4.551663,1.772586,3.263056,4.091980,3.090641,1.673430,1.523128,0.794987,2.256727,...,2.009858,2.576495,3.462176,2.572676,5.706240,2.498858,1.850879,NaN,4.143535,5.344525
GDSC:1006,3.171367,-2.014854,-1.818771,0.252367,-1.688469,0.020116,-0.460778,-0.877788,-0.103689,0.021624,...,-0.121731,-0.889294,0.681844,-2.041620,1.454613,-1.346782,-1.255548,NaN,2.067330,1.230528


In [15]:
pd.read_csv('data/Celline/genomic_expression_561celllines_697genes_demap_features.csv', index_col=0).head()

,LASP1,HOXA11,CREBBP,ETV1,GAS7,CD79B,PAX7,BTK,BRCA1,WAS,...,NCKIPSD,MTCP1,DDX3X,FANCG,SSX2,ETV5,CEBPA,LSM14A,CUX1,C15orf65
ACH-000828,9.393476,0.042644,3.935460,0.871844,0.070389,0.084064,0.000000,0.056584,3.339137,0.150560,...,4.071248,3.119356,6.849374,4.355439,0.000000,0.137504,1.769772,6.501598,4.700994,2.295723
ACH-000568,7.638074,0.056584,3.427606,0.201634,1.794936,0.739848,0.042644,0.333424,3.193772,0.815575,...,4.084064,4.634593,5.671576,5.525443,0.056584,2.195348,0.124328,5.811214,3.590961,1.550901
ACH-000560,5.728193,6.001352,5.032542,5.018812,0.432959,0.250962,0.000000,0.263034,4.678635,1.292782,...,5.057450,3.468583,6.617798,6.425761,0.000000,5.203201,1.922198,7.581351,5.320124,1.438293
ACH-000561,6.037163,1.565597,4.262283,0.790772,1.257011,0.028569,0.056584,0.042644,3.442280,0.286881,...,3.400538,3.407353,6.154211,4.794936,0.000000,3.984589,1.028569,6.533719,5.132166,2.144046
ACH-000562,7.050502,0.014355,3.360364,0.879706,0.084064,0.137504,0.000000,0.042644,4.939227,0.286881,...,4.125982,4.047015,6.281884,5.853497,0.056584,3.757023,0.056584,5.912171,4.877744,0.815575


In [16]:
pd.read_csv('data/Celline/genomic_methylation_561celllines_808genes_demap_features.csv', index_col=0).head()

,SKI_1_2159133_2160133,TNFRSF14_1_2486803_2487803,PRDM16_1_2984741_2985741,RPL22_1_6259679_6260679,CAMTA1_1_6844383_6845383,MTOR_1_11322608_11323608,PRDM2_1_14025734_14026734,PRDM2_1_14074875_14075875,CASP9_1_15850940_15851940,CASP9_1_15851285_15852285,...,BCORL1_X_129138163_129139163,ELF4_X_129244475_129245475,ELF4_X_129244688_129245688,GPC3_X_133119673_133120673,ATP2B3_X_152800579_152801579,FLNA_X_153603006_153604006,RPL10_X_153625405_153626405,RPL10_X_153625712_153626712,RPL10_X_153625837_153626837,MTCP1_X_154299547_154300547
ACH-000698,0.00000,0.00739,0.04799,0.14687,0.00553,0.54138,0.07160,0.00040,0.01836,0.03461,...,1.00000,0.00000,0.00000,0.020070,0.01706,0.16926,0.00145,0.00242,0.00242,0.01924
ACH-000489,0.00000,0.00363,0.64891,1.00000,0.00538,0.00000,0.00000,0.00000,0.00000,0.00000,...,1.00000,0.00000,0.00000,0.320580,0.72527,0.37572,0.00000,0.00000,0.00000,0.00000
ACH-000522,0.00143,0.47465,0.27715,0.19441,0.00858,0.40555,0.21625,0.01367,0.00338,0.00338,...,1.00000,0.18059,0.22562,0.054325,0.60830,0.65268,0.82345,0.59039,0.59039,0.07546
ACH-000613,0.00000,0.00000,0.05112,0.10686,0.00268,0.28618,0.07706,0.00000,0.00000,0.00000,...,0.01066,0.00000,0.00000,0.010660,0.53086,0.00918,0.00000,0.00448,0.00448,0.00000
ACH-000614,0.00118,0.00471,0.15165,0.07407,0.01050,0.51629,0.03292,0.00405,0.00000,0.00000,...,1.00000,0.07273,0.07273,0.106990,0.69472,0.28947,0.05354,0.04306,0.04306,0.00114


In [17]:
pd.read_csv('data/Celline/genomic_mutation_34673_demap_features.csv', index_col=0).head()

,TNFRSF14.1:2488170,TNFRSF14.1:2489805,TNFRSF14.1:2489824,TNFRSF14.1:2489856,TNFRSF14.1:2489868,TNFRSF14.1:2491328,TNFRSF14.1:2491335,TNFRSF14.1:2491373,TNFRSF14.1:2491403,TNFRSF14.1:2492117,...,MTCP1.23:154293902,MTCP1.23:154293909,MTCP1.23:154293951,MTCP1.23:154293978,MTCP1.23:154294034,MTCP1.23:154294203,MTCP1.23:154294207,MTCP1.23:154294278,MTCP1.23:154294279,MTCP1.23:154298967
ACH-001365,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ACH-001364,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ACH-001049,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ACH-001363,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ACH-001362,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
pd.read_csv('data/CCLE/CCLE_response.csv').head()

,DepMap_ID,pubchem,Z_SCORE,ActArea
0,ACH-000558,644241,-0.955879,0.0
1,ACH-000245,644241,-0.955879,0.0
2,ACH-000244,644241,-0.955879,0.0
3,ACH-000102,644241,-0.955879,0.0
4,ACH-000987,644241,-0.955879,0.0


In [20]:
pd.read_csv('data/CCLE/CCLE_smiles.csv').head()

,pubchem,isosmiles
0,11476171,C1CN(C1)CC2CC(C2)N3C=C(C4=C(N=CN=C43)N)C5=CC(=...
1,644241,CC1=C(C=C(C=C1)C(=O)NC2=CC(=CC(=C2)C(F)(F)F)N3...
2,6505803,C[C@H]1C[C@@H]([C@@H]([C@H](/C=C(/[C@@H]([C@H]...
3,10461815,CC1=C(NC(=C1C(=O)N2CCC[C@@H]2CN3CCCC3)C)/C=C\\...
4,208908,CS(=O)(=O)CCNCC1=CC=C(O1)C2=CC3=C(C=C2)N=CN=C3...


In [22]:
pd.read_csv('data/Drug/drug_threshold.csv').head()

,pubchem,IC50,DrugName,DrugID
0,9863776,-0.60596,(5Z)-7-Oxozeaenol,1242
1,3385,1.12360,5-Fluorouracil,179
2,10172943,-2.33850,A-443654,86
3,9549184,1.87310,A-770041,55
4,65110,6.19280,AICA Ribonucleotide,1001


In [23]:
pd.read_csv('data/Drug/1.Drug_listMon Jun 24 09_00_55 2019.csv').head()

,drug_id,Name,Synonyms,Targets,Target pathway,PubCHEM,Sample Size,Count
0,1242,(5Z)-7-Oxozeaenol,"5Z-7-Oxozeaenol, LL-Z1640-2",TAK1,"Other, kinases",9863776,945,266
1,179,5-Fluorouracil,5-FU,Antimetabolite (DNA & RNA),Other,3385,968,266
2,86,A-443654,KIN001-139,"AKT1, AKT2, AKT3",PI3K/MTOR signaling,10172943,425,266
3,55,A-770041,KIN001-111,"LCK, FYN","Other, kinases",9549184,426,266
4,1001,AICA Ribonucleotide,"AICAR, N1-(b-D-Ribofuranosyl)-5-aminoimidazole...",AMPK agonist,Metabolism,65110,872,266


In [24]:
pd.read_table('data/Drug/222drugs_pubchem_smiles.txt').head()

,25102847,COc1cc2nccc(Oc3ccc(NC(=O)C4(C(=O)Nc5ccc(F)cc5)CC4)cc3)c2cc1OC
0,148124,CC(=O)OC12COC1CC(O)C1(C)C(=O)C(O)C3=C(C)C(OC(=...
1,5289247,O=C1NC(=O)C(=Cc2ccc3nccnc3c2)S1
2,9868037,O=C(Nc1ccc(OC(F)(F)F)cc1)c1sccc1NCc1ccnc2ccccc12
3,11364421,CCC1C(=O)N(C)c2cnc(Nc3ccc(C(=O)NC4CCN(C)CC4)cc...
4,208908,CS(=O)(=O)CCNCc1ccc(-c2ccc3ncnc(Nc4ccc(OCc5ccc...


In [28]:
feat_mat, adj_list, degree_list = hkl.load('data/Drug/drug_graph_feat/10027278.hkl')

/Users/inouey2/miniconda3/envs/torch/lib/python3.10/site-packages/hickle/hickle.py:356: UserWarning: Input argument 'file_obj' appears to be a file made with hickle v3. Using legacy load...
  warnings.warn("Input argument 'file_obj' appears to be a file made"
